# 8기 과제 - 딥러닝 기반 상품 카테고리 자동 분류 서버

## 과제 개요
* 출제자 : 남상협 멘토 (justin@buzzni.com) / 버즈니 (http://buzzni.com) 대표
* 배경 : 카테고리 분류 엔진은 실제로 많은 서비스에서 사용되는 중요한 기계학습 기술이다. 본 과제의 주제는 버즈니 개발 인턴이자 마에스트로 6기 멘티가 아래와 나와 있는 기본 분류 모델을 기반으로 deep learning 기반의 feature 를 더해서 고도화된 분류 엔진을 만들어서 2016 한국 정보과학회 논문으로도 제출 했던 주제이다. 기계학습에 대한 학습과, 실용성 두가지 측면에서 모두 도움이 될 것으로 보인다.


## 과제 목표
* 입력 : 상품명, 상품 이미지
* 출력 : 카테고리
* 목표 : 가장 높은 정확도로 분류를 하는 분류 엔진을 개발



## 평가 항목 
* 성능평가 (100%)
 
## 제출 항목 
* 채점 서버에 자신이 분류한 class id 리스트를 파라미터로 넣어서 호출한다. 
* name - 자신의 이름을 넣는다. 실제 점수판에는 공개가 안됨, 추후 평가시에 일치하는 이름의 멘티 점수로 사용함. 요청한 평가 중에서 가장 높은 점수의 평가 점수로 업데이트됨.
* nickname - 점수판에 공개되는 이름, 자신의 이름으로 해도 되고, 닉네임으로 해도 됨. 구분을 위해서 사용하는 feature(text, textimage) 와 알고리즘 (svm, cnn) 등을 닉네임 뒤에 붙여준다. 
* pred_list - 분류한 카테고리 id 리스트를 , 로 묶은 데이터 
* 평가 점수가 반환된다. - precision, 높을 수록 좋다. 두가지 방법 각각 50%씩 점수 반영 
* mode - 'test' 로 호출하면 웹으로 순위가 공개되는 테스트 평가를 수행하고 결과 점수가 반환된다. 해당 결과 점수는 http://eval.buzzni.net:20002/score 에서 확인 가능함. 실제 성적 평가는 'eval' 로 평가용 데이터로 호출하면 된다. 이때는 점수가 반환되거나, 웹 점수 보드에도 나오지 않는다. 
* 너무 자주 평가를 요청하기 보다, 가급적 자체적으로 평가 해서, 괜찮게 나올때 요청하길 권장 
```python
import requests
name='test1'
nickname='test1_text_svm'
mode='test' #'eval' 을 실제 성적 평가용. 분류 점수 반환 안됨.
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list)),
         'name':name,'nickname':nickname,"mode":mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)
print (d.json())         
         ```

## 성능 향상 포인트
* http://localhost:8000/notebooks/maestro8_deeplearning_product_classifier.ipynb 이 노트북에 있는 딥러닝 기반의 분류기로 분류할 경우에 더 높은 성능을 낼 수 있어서 유리함
* 아래의 방법들은 하나의 예이고, 아래에 나와 있지 않은 다양한 방법들도 가능함.
* 전처리 
 * 오픈된 형태소 분석기(예 - konlpy) 를 써서, 단어 띄어쓰기를 의미 단위로 띄어서 학습하기
 * bigram, unigram, trigram 등 단어 feature 를 더 다양하게 추가하기
* 딥러닝 
 * embedding weight 를 random 이 아닌 학습된 값을 사용하기 (https://radimrehurek.com/gensim/models/word2vec.html)
 * 이미지 feature 를 CNN으로 추출할때 더 성능이 좋은 모델 사용하기 (예제로 준 데이터는 mobilenet 으로 성능보다 속도 위주로 된 모델)
 * 다양한 파라미터(hyper parameter) 로 실험 해보기 
* 피쳐 조합  
 * 이미지 feature 와 text feature 를 합치는 부분 잘하기 


## 평가 점수 서버 
* 현재 평가 순위를 json 형태로 반환한다.
* 여러번 호출했을때는 가장 높은 점수로 업데이트 한다.
 * http://eval.buzzni.net:20002/score
* 실제 점수는 

In [15]:
import sys
from sklearn.externals import  joblib
from sklearn.grid_search import GridSearchCV
from sklearn.svm import  LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import  TfidfVectorizer,CountVectorizer
import os
import numpy as np
import string
from keras import backend
from keras.layers import Dense, Input, Lambda, LSTM, TimeDistributed
from keras.layers.merge import concatenate
from keras.layers.embeddings import Embedding
from keras.models import Model



### 파일에서 학습 데이터를 읽는다.

In [16]:
import json


In [17]:
x_text_list = []
y_text_list = []
enc = sys.getdefaultencoding()
with open("refined_category_dataset.dat",encoding=enc) as fin:
    for line in fin.readlines():
#         print (line)
        info = json.loads(line.strip())
        x_text_list.append((info['pid'],info['name']))
        y_text_list.append(info['cate'])
        

In [18]:
# joblib.dump(y_name_id_dict,"y_name_id_dict.dat")

### text 형식으로 되어 있는 카테고리 명을 숫자 id 형태로 변환한다.

In [19]:
y_name_id_dict = joblib.load("y_name_id_dict.dat")

In [20]:
print(y_name_id_dict)

{'출산/육아': 16, '도서/문구': 8, '뷰티': 0, '가구/인테리어': 11, '반려동물': 4, '취미': 5, '잡화': 14, '디지털': 10, '생필품/주방': 13, '건강': 7, '자동차/공구': 1, '식품': 6, '스포츠/레저': 9, '컴퓨터': 3, '가전': 12, '의류': 2, '여행/e쿠폰': 15}


In [21]:

# y_name_set = set(y_text_list)
# y_name_id_dict = dict(zip(y_name_set, range(len(y_name_set))))
# print(y_name_id_dict.items())
# y_id_name_dict = dict(zip(range(len(y_name_set)),y_name_set))
y_id_list = [y_name_id_dict[x] for x in y_text_list]


### text 형태로 되어 있는 상품명을 각 단어 id 형태로 변환한다.

In [22]:
from sklearn.model_selection import train_test_split

vectorizer = CountVectorizer()
x_list = vectorizer.fit_transform(map(lambda i : i[1],x_text_list))
y_list = [y_name_id_dict[x] for x in y_text_list]
tmp_list = list(map(lambda i : i[1],x_text_list))
print(tmp_list[0:10])

['#MD추천정품# 심플베드 엘레강스 고급형 접이식침대 더블사이즈115cm/수동/등받이6단으로 원하는각도조절', '(9731200) 소형제도판 (S) 661 A3', '(AGCRIP 웨빙 포켓조끼 빅사이즈 조끼 MK321_324 아웃', '(ATK아텍스 전동접이식침대(BE556) 전동침대 간병용침', '(MST플래그 메세지 캔들 마더 향초 양초캔들 캔들선물 아로마캔들 향초캔들', '(광일체어) 스카치A형 3인 등무 장의자', '(광일체어) 스파고아우드 2인 등유 로비용장의자', '(라움스튜디오)스크류 블랙봉 (일반-2.5cm) / 블랙 커튼봉/ 25mm / 일반봉 / 커튼봉 / 블랙', '(모슬리메모리즈) 산토리니 브리즈 (중) 모슬리메모리즈 아로마캔들 디퓨저 소이캔들 소이왁스', '(무료배송)양키캔들 악세사리 윅디퍼 심지가위 라이터']


In [23]:
nouns_list=[]
from konlpy.corpus import kolaw
from konlpy.tag import *
kkma=Kkma()
for i in range(0,len(tmp_list)):
    nouns=kkma.nouns(tmp_list[i])
    str1=""
    for N in nouns:
        for k in range(0,len(N)):
            str1+=N[k]
        str1+=' '
    nouns_list.append((x_text_list[i][0],str1))
print(nouns_list[0:10])

[('1234621373', '추천 추천정품 정품 심플 심플베드 베드 엘레강스 고급 고급형 형 접이 접이식침대 식 침대 더블 더블사이즈115 사이즈 115 수동 등받이 등받이6단 6 단 각도 각도조절 조절 '), ('1543513007', '9731200 소형 소형제도판 제도 판 661 3 '), ('1546650266', '웨빙 포켓 포켓조끼 조끼 빅 빅사이즈 사이즈 321 324 아웃 '), ('1589443530', '텍스 전 전동접이식침대 동접 이식 침대 556 전동침대 동 간병 침 '), ('1520510486', '플래그 메세지 마 향초 양초 양초캔 캔 선물 아로마 아로마캔 향초캔 '), ('1512822613', '광일 광일체어 체어 스카치 형 3 3인 인 등 등무 무 장의자 '), ('1525985492', '광일 광일체어 체어 스파 우드 2 2인 인 등유 로비 장의자 '), ('1326388222', '라 라움스튜디오 움 스튜디오 스크류 블랙 블랙봉 봉 일반 일반-2.5 -2.5 커튼 커튼봉 25 일반봉 '), ('1426607155', '모슬 모슬리메모리즈 리 메모리 즈 산토리 브리즈 중 아로마 아로마캔 캔 디퓨저 소이 소이캔 소이왁스 왁스 '), ('1334473563', '무료 무료배송 배송 양키 양키캔 캔 악세사리 윅 윅디 디 심지 심지가위 가위 라이터 ')]


### train test 분리하는 방법 

In [24]:
X_train, X_test , y_train, y_test = train_test_split(nouns_list, y_list, test_size=0.1, random_state=42)
print(X_test)

[('1494008925', '백색 색 센스 1 에나멜 페인트 목재 철재 조색 '), ('1369212821', '몰리 몰리즈 즈 세라 런던 런던블루토파즈 블루 토파즈 세트 72212 14 '), ('1235527338', '애 프 프렌즈 렌즈 발코니 발코니매트 매트 폭 1.5 1.8 선택 다용도 다용도매트 베란다 베란다매트 거실 거실매트 방습 냉기 냉기차단 차단 '), ('1607501430', '2060 다이 다이마 마 스펜더스 다이마루스커트 마루 스커트 서 '), ('1545024003', '숲 적색 멀티 멀티페인트 페인트 4 수성 수성페인트 '), ('1620878204', '새 새간식 간식 미네랄 블록 오렌지 영양 영양간식 부리 부리연마 연마 새장 '), ('1279440620', '카 카마운트밴드 마운트 밴드 홀더 거치대 차량 '), ('1570090582', '카 카브리오 브리 오 자동차 자동차리스 리스 마 마세라티리스 세라 티 장기 장기렌트비용 렌트 비용 '), ('1518758452', '애견 애견껌 껌 화이트 스틱 4 3 3개 개 강아지 강아지껌 '), ('1654570057', '냉장 별가 순 순쌀떡3 쌀 떡 3 5 '), ('1104082461', '큐 큐티 티 곰 16 펜 펜브로치 브로치 곳 캐릭터 펜패 패 전통 부자재 퀼트 퀼트부자재 펠트 '), ('1486851839', '3.1 젠더 5 20 '), ('1041691443', '저 록스 255 흑백 흑백레이저프린터 레이저 프린터 토너 토너포함 포함 128 30 자동 자동양면인쇄 양면 인쇄 무선 무선네트워크지원 네트워크 지원 '), ('1382443280', '레 레페넥스트랩 페 넥스트 랩 전기종 전기종호환 호환 화이트 화이트로즈 로즈 소니 5000 '), ('1561074017', '그린 그린핑거 핑거 순 자연 자연보습 보습 베이비 비누 '), ('1367822110', '멀티 고무 고무후드 후드 62 범용 접이 접이식 식 '), ('1114102714', '애 손잡이 손잡이필통 필

In [25]:
# print(vectorizer.transform(list(map(lambda i : i[1],X_train))))

### 몇개의 파라미터로 간단히 테스트 하는 방법

In [26]:

for c in [0.1]:
    clf = LinearSVC(C=c)
    X_train_text = map(lambda i : i[1],X_train)
    clf.fit(vectorizer.transform(X_train_text), y_train)
    print (c,clf.score(vectorizer.transform(map(lambda i : i[1],X_test)), y_test))


0.1 0.678823529412


#### 평가 데이터에 대해서 분류를 한 후에  평가 서버에 분류 결과 전송

In [27]:
eval_x_text_list = []
with open("soma8_test_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list.append((info['pid'],info['name']))


In [28]:
tmp_list2=list(map(lambda i : i[1],eval_x_text_list))

test_nouns=[]
for i in range(0,len(tmp_list2)):
    nouns=kkma.nouns(tmp_list2[i])
    str1=""
    for N in nouns:
        for k in range(0,len(N)):
            str1+=N[k]
        str1+=' '
    test_nouns.append((eval_x_text_list[i][0],str1))
print(test_nouns[0:10])

[('720988653', '스프레이 기아 오피 오피러스 러스 8 라이트 라이트메탈 메탈 주문 주문제품 제품 15 15일예상 일 예상 '), ('1378489203', '엑스포 엑스포넷주식회사 넷 주식 주식회사 회사 코일 코일탭 탭 162.0 18.6 1 절삭 절삭공구 공구 작업 작업공구 배송 '), ('1621140511', '오가 오가닉맘 닉 맘 펭 펭코 코 목도리 7 02 인천 인천점 점 '), ('1549548735', '골프 골프거리측정기 거리 측정기 골프거리 측정 망원경 '), ('1642215271', '극 극동고성능해빙기 동고 성능 해빙기 고성능 고성능전기해빙기 전기 10 '), ('1594181530', '인두 플라이 드라이버 드라이버등 등 공구 공구세트 세트 수공구 '), ('1002465039', '토이 토이트론 트론 실 실바 바 안 안패밀리 패밀리 유치원 유치원학예회세트 학예회 세트 3589 '), ('1592587497', '누 누리 리 사이즈 뉴 뉴테크 테크 좌식 바베큐 '), ('181355585', '국내산 혀 혀클리너 클리너 1 1개 개 낱개 크리너 입 입냄새 냄새 제거 구강 청결제 여행용 혀간 간 칫솔 세트 치 치실 실 치약 백태 '), ('1212013675', '추천 철사 철사밴드 밴드 40 통 100 호스 호스밴드 호스조임 조 임 조임쇠 파이프 파이프밴드 철 철밴드 ')]


In [29]:
pred_list = clf.predict(vectorizer.transform(map(lambda i : i[1],test_nouns)))

In [30]:
# print (pred_list.tolist())

In [31]:
import requests
name='test0'
nickname='test0_text_svm'
mode='test'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


{'precision': 0.7345201238390093, 'msg': 'If you pull docker image before 2017-09-27 21:30,  pull your docker image again.'}


#### eval 데이터에 대해서 분류를 한 후에  평가 서버에 분류 결과 전송
 * 실제 여기서 나온 점수로 채점을 한다.

In [40]:
eval_x_text_list2 = []
with open("soma8_eval_data.dat",encoding=enc) as fin:
    for line in fin.readlines():
        info = json.loads(line.strip())
        eval_x_text_list2.append((info['pid'],info['name']))
        
tmp_list3=list(map(lambda i : i[1],eval_x_text_list2))

eval_nouns=[]
for i in range(0,len(tmp_list3)):
    nouns=kkma.nouns(tmp_list3[i])
    str1=""
    for N in nouns:
        for k in range(0,len(N)):
            str1+=N[k]
        str1+=' '
    eval_nouns.append((eval_x_text_list2[i][0],str1))
print(eval_nouns[0:10])
pred_list2 = clf.predict(vectorizer.transform(map(lambda i : i[1],eval_nouns)))

print(pred_list2[0:10])

[('1649273909', '실버 실버카 카 보행 토모 토모카 레드 레드걸음보조기 걸음 보조기 노인 '), ('1531800813', '프리 프리메라 메라 베이비 선 쿠션 15 품 '), ('1596516044', '컬러 730 3 1 '), ('1524822010', '앙쥬 욕실 미끄럼 미끄럼방지 미끄럼방지패드 방지 패드 바다 바다이야기 이야기 '), ('1553648730', '핸드 핸드피싱 피 싱 물방울 물방울추 추 유동 유동봉돌 봉돌 도래 도래봉돌 해결사 해결사채비 채비 '), ('1303364322', '캐논 정품 3 바디 22 렌즈 최신 최신상품 상품 셀 셀카기능 카 기능 미러 미러리스카메라 리스 카메라 '), ('1647212375', '채혈침 란셋 사 사혈침 혈침 28 200 200입 입 1 1박스 박스 '), ('1169936535', '현대 현대백화점 백화점 압소 압소바 바 주 트상 트상하복 하복 '), ('1628287970', '트릭 트릭시 시 고양 고양이하네스세트 이하 네스 세트 블루 블루줄무늬 줄무늬 야광 고양이 고양이목줄 목줄 '), ('1605794098', '창고 슬림 슬림더블지퍼 더블 지퍼 장 장지갑 지갑 카드 카드지갑 티 남자 카 ')]
[ 7 16 12  9 13 10  7  2  4 14]


In [41]:

name='test02'
nickname='test0_text_svm'
mode='eval'
param = {'pred_list':",".join(map(lambda i : str(int(i)),pred_list2.tolist())),
         'name':name,'nickname':nickname,'mode':mode}
d = requests.post('http://eval.buzzni.net:20001/eval',data=param)

print (d.json())


{'precision': '', 'msg': 'success'}
